# Matrix completion via recommendation system example

This example demonstrates the use of matrix completion techniques on a recommendation system.  The recommendation system uses data from the [360K Last.fm dataset](http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html).

In [ ]:
#!pip install -U implicit h5py

^C


In [2]:
# retrieving last.fm dataset
from implicit.datasets.lastfm import get_lastfm
import numpy as np
import pandas as pd
from scipy import sparse
import os
from pathlib import Path

## Downloading and saving the Last.fm dataset

In [3]:
filepath = r'datasets/'
Path(filepath).mkdir(exist_ok=True)

if not os.path.exists(filepath + r'artist_user_plays.npz'):
    # save our dataset in sparse format
    artists, users, artist_user_plays = get_lastfm()

    sparse.save_npz(filepath + r'artist_user_plays.npz', artist_user_plays)
    np.save(filepath + 'artists.npy', artists)
    np.save(filepath + 'users.npy', users)
else:
    # load our dataset into original format
    artist_user_plays = sparse.load_npz(filepath + r'artist_user_plays.npz')
    artists = np.load(filepath + 'artists.npy', allow_pickle=True)
    users = np.load(filepath + 'users.npy', allow_pickle=True)

184MB [00:07, 24.3MB/s]                              


In [ ]:
# some samples of artists
artists[np.random.randint(size=50, low=0, high=len(artists))]

# some samples of users
users[0]

array(['jaco', 'willeke alberti', 'tyken feat. awa', 'nordman',
       'poumi lescaut', 'muga', 'theatre brook', 'cinema playground',
       'marc van linden & matys', 'man made', 'arise and ruin',
       'dennis ferrer', 'finbar furey', 'john williams & dean magraw',
       'pascale picard', 'me 1st & the ggs', 'limmy', 'plutonium orange',
       'caliko', 'josÉ josÉ', 'layo and bushwaca',
       'hüsnü senlendirici & laço tayfa', 'butch vig', 'dj grégory',
       'king and queen', 'el mato a un policia motorizado',
       'david donoghue', 'the new government', 'mirra+fay',
       'soundshifter', 'pat thomas', 'samantha moore', 'liam shachar',
       'someone', 'but & memo', 'the eclectics',
       'freud marx engel ja jung', 'hybryds', 'becky schlegel', 'decapod',
       'baron zen', 'chaos factory', 'agrotoxico', 'stone the crows',
       'franz \x81hmad cow\x81h tornado and the yamanba-gals',
       'nico & tetta', 'kim gordon', 'boy needle', 'miglo', 'john wesley'],
      dtype=o

In [1]:
# return the dimensions of data
artists.shape, users.shape, artist_user_plays.shape

NameError: name 'artists' is not defined

In [ ]:
artist_user_plays

In [ ]:
# return the number of entries 
artist_user_plays.count_nonzero() 

In [ ]:
# investigate the proportion of non-zero entries
artist_user_plays.count_nonzero()  / (292385 * 358868)

## Preparing the data
Okapi BM25 (Best Matching) scoring is a ranking algorithm used by search engines to estimate the relevance of items to a given search query, based on the frequency of occurrences and the size of the reference pool.  The origin of the algorithm is used in search terms in a pool of documents.

For completeness, the BM25 score of query $Q=\{q_1, \ldots, q_n\}$ for a document $D$ is calculated as:

$$\text{BM25}(D, Q) = \sum_{i=1}^{n} \frac{IDF(q_i) \cdot f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgD}})},$$
where
- $IDF(q_i)$ is the inverse document frequency of term $q_i$.
- $f(q_i, D)$ is the term frequency of $q_i$ in the document $D$.
- $k_1$ and $b$ are parameters controlling term saturation and document length normalization.
- $D$ is the length of the document.
- $\text{avgD}$ is the average document length in the corpus.


In [ ]:
from implicit.nearest_neighbours import bm25_weight

# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

## Training the model with alternating least squares

In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=16, regularization=0.05, alpha=2.0)
model.fit(user_plays)

## Similar artists recommendation

In [ ]:
list(artists).index('beethoven')

In [ ]:
artist_id = 42883
ids, scores = model.similar_items(artist_id)

In [ ]:
pd.DataFrame({"artist": artists[ids], "score": scores})

## User-specific recommendation

In [ ]:
userid = 10
ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)

In [ ]:
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.isin(ids, user_plays[userid].indices)})